## 4.5 학습 알고리즘 구현하기

**전제**
- 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 합니다. 신경망 학습은 다음과 같이 4단계로 수행합니다.

**1단계 - 미니배치**
- 훈련 데이터 중 일부를 무작위로 가져옵니다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표입니다.

**2단계 - 기울기 산출**
- 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구합니다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시합니다.

**3단계 - 매개변수 갱신**
- 가중치 매개변수를 기울기 방향으로 아주 조금 갱신합니다.

**4단계 - 반복**
- 1~3단계를 반복합니다.

위 과정이 신경망 학습이 이뤄지는 순서입니다. 이는 경사 하강법으로 매개변수를 갱신하는 방법이며, 이때 데이터를 미니배치로 무작위로 선정하기 때문에 **확률적 경사 하강법(stochastic gradient descent, SGD)** 이라고 부릅니다.

## 4.5.1 2층 신경망 클래스 구현하기

In [14]:
import sys, os
sys.path.append(os.pardir)
from deep_learning_github.common.functions import *
from deep_learning_github.common.gradient import numerical_gradient
import numpy as np

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): # std : 표준편차
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x, t)
        print("test")
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads       
        

In [15]:
print( np.random.randn(3, 2))
0.01 * np.random.randn(3, 2)

[[-0.65265541 -0.60224536]
 [ 0.29948305  1.02341424]
 [ 1.11735057 -1.79612884]]


array([[ 0.00753859,  0.00621727],
       [ 0.0157265 ,  0.01134564],
       [-0.00537191,  0.00268314]])

In [16]:
 np.zeros(3)

array([0., 0., 0.])

In [17]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape )
print(net.params['b1'].shape )
print(net.params['W2'].shape )
print(net.params['b2'].shape )

(784, 100)
(100,)
(100, 10)
(10,)


In [18]:
x = np.random.rand(100, 784) # 더미 입력 데이터(100장 분량)
y = net.predict(x)

In [23]:
x = np.random.rand(100, 784) # 더미 입력 데이터
t = np.random.rand(100, 10) # 더미 정답 레이블

grads = net.numerical_gradient(x, t)

print(grads['W1'].shape )
print(grads['b1'].shape )
print(grads['W2'].shape )
print(grads['b2'].shape )

(784, 100)
(100,)
(100, 10)
(10,)


## 미니배치 학습 구현하기

미니배치 학습이란?  
훈련 데이터 중 일부를 무작위로 꺼내고(미니배치), 그 미니배치에 대해서 경사법으로 매개변수를 갱신

In [28]:
import numpy as np
sys.path.append(os.pardir)
from deep_learning_github.dataset.mnist import load_mnist
# from deep_learning_github.ch04.two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# 하이퍼 파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) # 성능 개선판!
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    print("loss:", loss)
    train_loss_list.append(loss)
    

loss: 2.295637978603324
loss: 2.284099118644126


KeyboardInterrupt: 

## 4.5.3 시험 데이터로 평가하기

위의 학습을 반복함으로써 손실 함수의 값이 서서히 내려가는 것을 확인할 수 있다.  
이때의 **손실 함수 값이란,** 정확히는 **'훈련 데이터의 미니배치에 대한 손실 함수'** 의 값이다.  
훈련 데이터의 손실 함수 값이 작아지는 것은 신경망이 잘 학습하고 있다는 방증이지만, 이 결과만으로는 다른 데이터셋에도 비슷한 실력을 발휘할지는 확실하지 않다.


즉 **'오버피팅'** 을 일으키지 않는지 확인해야 한다.  
오버피팅이란, 예를 들어 훈련 데이터에 포함된 이미지만 제대로 구분하고, 그렇지 않은 이미지는 식별할 수 없다는 뜻이다.

In [36]:
max(train_size/ batch_size, 1) # 600.0

600.0

In [34]:
import numpy as np
sys.path.append(os.pardir)
from deep_learning_github.dataset.mnist import load_mnist
# from deep_learning_github.ch04.two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)


# 하이퍼 파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1 epoch당 반복 수
iter_per_epoch = max(train_size/ batch_size, 1) # 600.0


network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) # 성능 개선판!
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1 epoch 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        print("train acc, test acc : " + str(train_acc) + ", " + str(test_acc))
        

train acc, test acc : 0.11236666666666667, 0.1135


KeyboardInterrupt: 

In [37]:
for i in range(iters_num):
    if i % iter_per_epoch == 0:
        print(i)

0
600
1200
1800
2400
3000
3600
4200
4800
5400
6000
6600
7200
7800
8400
9000
9600


In [39]:
for i in range(10):
    a = np.random.choice(100, 10)
    print(a)

[ 7 80  1  0 90 99 94 35 95 56]
[76  6 56 91 69 95 33 96 85 42]
[ 2 33 12 26 48 91  8 26  0 24]
[43 18 77 21 53 55 63  1 43 81]
[52 55 65 56 36 24 82 57 14 50]
[71 64 29 77 99 15 70 22 61 57]
[66 33 65 64 34  9 35 57 13 92]
[39 68 65  8 77 39 43 11 53 15]
[92 77 51 24 59  8 88 82  4 26]
[ 4 89 48  5  1 55 28 49 68 37]
